In [1]:

import time
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.utils import uri_helper
from cflib.crazyflie.swarm import CachedCfFactory
from cflib.crazyflie.swarm import Swarm
import os

In [2]:
# Initial positions of drones: 1, 2 ...., 7
# x    y    z  
z_altitude = 1.5  # Altitude for the drones
# positions = [
#     (0.15, 1.33, z_altitude),
#     (0.94, 1.0, z_altitude),
#     (2.03, 1.33, z_altitude),
#     (2.03, 2.66, z_altitude),
#     (0.94, 3.0, z_altitude),
#     (0.15, 2.66, z_altitude),
#     (0.94, 2.0, z_altitude),
# ]
positions = [
    (0.35, 1.33, z_altitude),

    (1.5, 1.33, z_altitude),

    (1.5, 2.66, z_altitude),

    (0.35, 2.66, z_altitude),
]
# uris_list = [
#     'radio://0/80/2M/E7E7E7E701',
#     'radio://0/80/2M/E7E7E7E708',
#     'radio://0/80/2M/E7E7E7E703',
#     'radio://1/80/2M/E7E7E7E704',
#     'radio://1/80/2M/E7E7E7E705',
#     'radio://1/80/2M/E7E7E7E706',
#     'radio://2/80/2M/E7E7E7E707',
# ]

uris_list = [
    'radio://0/80/2M/E7E7E7E701',
    'radio://0/80/2M/E7E7E7E703',
    'radio://1/80/2M/E7E7E7E704',
    'radio://1/80/2M/E7E7E7E706',
]
filename = 'C:/Users/omerr/drone_win/data_xyz/hexagon_xyz.csv'
figurename = 'hexagon_xyz.jpg'
gifname = 'hexagon_xyz.gif'

# 1. Define PID values for each drone (example values, adjust as needed)
# pid_dict = {
#     'radio://0/80/2M/E7E7E7E701': (2.02, 0.025, 0.7),
#     'radio://0/80/2M/E7E7E7E708': (2.02, 0.025, 0.7),
#     'radio://0/80/2M/E7E7E7E703': (2.02, 0.025, 0.7),
#     'radio://1/80/2M/E7E7E7E704': (2.02, 0.025, 0.7),
#     'radio://1/80/2M/E7E7E7E705': (2.02, 0.025, 0.7),
#     'radio://1/80/2M/E7E7E7E706': (2.02, 0.025, 0.7),
#     'radio://2/80/2M/E7E7E7E707': (2.02, 0.025, 0.7),
# }

pid_dict = {
    'radio://0/80/2M/E7E7E7E701': (2.02, 0.025, 0.7),
    'radio://0/80/2M/E7E7E7E703': (2.02, 0.025, 0.7),
    'radio://1/80/2M/E7E7E7E704': (2.02, 0.025, 0.7),
    'radio://1/80/2M/E7E7E7E706': (2.02, 0.025, 0.7),
}


# 2. Function to set PID for a single drone
def set_pid(scf, pid_vals):
    zKp, zKi, zKd = pid_vals
    scf.cf.param.set_value('posCtlPid.zKp', zKp)
    scf.cf.param.set_value('posCtlPid.zKi', zKi)
    scf.cf.param.set_value('posCtlPid.zKd', zKd)
    # Add X/Y PID if needed


In [3]:
def take_off(scf):
    commander= scf.cf.high_level_commander

    commander.takeoff(z_altitude, 5.0)
    time.sleep(2)

def land(scf):
    commander = scf.cf.high_level_commander

    commander.land(0.0, 5.0)
    time.sleep(2)

    commander.stop()


def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            # print("{} {} {}".
            #       format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break


def reset_estimator(scf):
    cf = scf.cf
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')

    wait_for_position_estimator(cf)


def position_callback(timestamp, data, logconf):
    x = data['kalman.stateX']
    y = data['kalman.stateY']
    z = data['kalman.stateZ']

    with open(filename, 'a') as f:
        f.write('{},{},{},{}\n'.format(timestamp, x, y, z))


def start_position_printing(scf):
    log_conf = LogConfig(name='Position', period_in_ms=100)
    log_conf.add_variable('kalman.stateX', 'float')
    log_conf.add_variable('kalman.stateY', 'float')
    log_conf.add_variable('kalman.stateZ', 'float')

    scf.cf.log.add_config(log_conf)
    log_conf.data_received_cb.add_callback(position_callback)
    log_conf.start()


def run_sequence_setpoint(scf, position):
    cf = scf.cf
    x, y, z = position
    print(f"Moving to position: {position}")
    for _ in range(100):  # Send setpoint for 3 seconds (30*0.1s)
        cf.commander.send_position_setpoint(x, y, z, 0)
        time.sleep(0.1)

    for _ in range(25):  # Send setpoint for 3 seconds (30*0.1s)
        cf.commander.send_position_setpoint(x, y, 1.0, 0)
        time.sleep(0.1)

    for _ in range(25):  # Send setpoint for 3 seconds (30*0.1s)
        cf.commander.send_position_setpoint(x, y, 0.25, 0)
        time.sleep(0.1)

    cf.commander.send_stop_setpoint()
    cf.commander.send_notify_setpoint_stop()

def run_sequence_landing(scf, position):
    cf = scf.cf
    x, y, z = position
    print(f"Landing to position: {position}")
    for _ in range(50):  # Send setpoint for 3 seconds (30*0.1s)
        cf.commander.send_position_setpoint(x, y, 0.5, 0)
        time.sleep(0.1)
    cf.commander.send_stop_setpoint()
    cf.commander.send_notify_setpoint_stop()
    time.sleep(0.1)



In [4]:

cflib.crtp.init_drivers()

factory = CachedCfFactory(rw_cache='./cache')
with Swarm(uris_list, factory=factory) as swarm:

    if os.path.exists(filename):
        os.remove(filename)

    # print('Setting PID values for each drone')
    # pid_args = {uri: (pid_dict[uri],) for uri in uris_list}
    # swarm.parallel_safe(set_pid, args_dict=pid_args)
    # print('PID values set')

    print('Resetting estimator')
    swarm.reset_estimators()
    print('Estimator reset')

    print('Taking off')
    swarm.parallel_safe(take_off)
    time.sleep(2.0)

    print('Moving to positions')
    # Map each URI to its position
    args_dict = {uri: (pos,) for uri, pos in zip(uris_list, positions)}
    swarm.parallel_safe(run_sequence_setpoint, args_dict=args_dict)
    print('Landing')
    # swarm.parallel_safe(run_sequence_landing, args_dict=args_dict)
    # time.sleep(1.0)
    # swarm.parallel_safe(land)

Resetting estimator
Estimator reset
Taking off
Moving to positions
Moving to position: (0.35, 1.33, 1.5)
Moving to position: (1.5, 1.33, 1.5)
Moving to position: (1.5, 2.66, 1.5)
Moving to position: (0.35, 2.66, 1.5)
Landing


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

print("Plotting and saving data...")
df = pd.read_csv(f'C:/Users/omerr/drone_win/data_xyz/pid_xyz_pz{zKp}.csv', header=None, names=['timestamp','x', 'y', 'z'])
df['time_s'] = (df['timestamp'] - df['timestamp'].iloc[0]) / 1000.0  # adjust divisor if needed

fig, axes = plt.subplots(1, 3, figsize=(20, 7), sharex=True)

coords = ['x', 'y', 'z']
labels = ['X Position (m)', 'Y Position (m)', 'Z Position (m)']
titles = ['Drone X Over Time', 'Drone Y Over Time', 'Drone Z Over Time']

for i, (coord, label, title) in enumerate(zip(coords, labels, titles)):
    axes[i].plot(df['time_s'], df[coord], label=f'{coord.upper()} Position')
    mean_val = df[coord].mean()
    var_val = df[coord].var()
    axes[i].set_xlabel('Time (s)')
    axes[i].set_ylabel(label)
    axes[i].set_title(title)
    axes[i].grid()
    axes[i].legend()
    # Annotate mean and variance
    axes[i].text(
        0.5, 0.95,
        f'Mean: {mean_val:.3f} m\nVar: {var_val:.1e}',
        transform=axes[i].transAxes,
        fontsize=12,
        color='darkred',
        ha='center',
        va='top',
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
    )
axes[2].axhline(y=target_z, color='red', linestyle='--', linewidth=2, label='Setpoint = 1.5m')

plt.tight_layout()
plt.savefig(f'C:/Users/omerr/drone_win/data_img/{figurename}', dpi=300, bbox_inches='tight')
plt.show()

Plotting and saving data...


NameError: name 'zKp' is not defined